In [60]:
import yfinance as yf
import numpy as np

tickers = {}
ticker_names = ["BTC-EUR", "SOL-EUR", "ETH-EUR"]
for ticker in ticker_names:
    tickers[ticker] = yf.download(ticker, start='2021-07-20', end='2024-10-30')
    tickers[ticker].reset_index(inplace=True)

print(tickers['BTC-EUR'])

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price                       Date     Adj Close         Close          High  \
Ticker                                 BTC-EUR       BTC-EUR       BTC-EUR   
0      2021-07-20 00:00:00+00:00  25299.851562  25299.851562  26284.099609   
1      2021-07-21 00:00:00+00:00  27227.458984  27227.458984  27762.083984   
2      2021-07-22 00:00:00+00:00  27447.947266  27447.947266  27691.796875   
3      2021-07-23 00:00:00+00:00  28527.830078  28527.830078  28527.830078   
4      2021-07-24 00:00:00+00:00  29132.255859  29132.255859  29300.416016   
...                          ...           ...           ...           ...   
1193   2024-10-25 00:00:00+00:00  62971.476562  62971.476562  63552.078125   
1194   2024-10-26 00:00:00+00:00  61702.476562  61702.476562  63418.800781   
1195   2024-10-27 00:00:00+00:00  62047.167969  62047.167969  62327.910156   
1196   2024-10-28 00:00:00+00:00  62920.054688  62920.054688  63195.246094   
1197   2024-10-29 00:00:00+00:00  64624.406250  64624.406250  64

In [61]:
#Generate color palette from magma palette, excluding dark colors
from bokeh.palettes import Magma256
color_codes = [int(num) for num in np.linspace(100,255, len(ticker_names))]
colors = {k: v for k, v in zip(ticker_names, [Magma256[color] for color in color_codes])}
colors

{'BTC-EUR': '#892881', 'SOL-EUR': '#F56C5B', 'ETH-EUR': '#FBFCBF'}

In [62]:
import numpy as np
from bokeh.plotting import figure, output_file, show, column
from bokeh.io import curdoc
from bokeh.models import HoverTool, ColumnDataSource, RangeTool

#Set dark theme to fit with magma palette
curdoc().theme='dark_minimal'

#The file to save the model
output_file("test.html")

#Instantiate figure object
dates = np.array(tickers['BTC-EUR']['Date'], dtype=np.datetime64)
graph = figure(height=500, width=1000, x_axis_type = "datetime", title = "Relative price changes", 
               tools='tap,xpan,reset', x_axis_location = "above", x_range =(dates[-365], dates[-1])) 

select = figure(title="Drag the middle and edges of the selection box to change the range",
                height=150, width=1000, y_range=graph.y_range, x_axis_type="datetime",
                tools="", toolbar_location=None)
range_tool = RangeTool(x_range=graph.x_range)
range_tool.overlay.fill_color = "#900C3F"
range_tool.overlay.fill_alpha = 0.2


#Names axes
graph.xaxis.axis_label = 'Date'
graph.yaxis.axis_label = 'Relative Price'

#Plotting the line graph

renderers = []
for ticker_name in ticker_names:
    tickers[ticker_name]['RelativeChange'] = tickers[ticker_name].Close / tickers[ticker_name].Close.iloc[0]
    y_axis_coordinates = tickers[ticker_name]['RelativeChange']
    tickers[ticker_name]['Legend_label'] = ticker_name

    source = ColumnDataSource(tickers[ticker_name])
    color = colors[ticker_name]
    l = graph.line('Date_', 'RelativeChange_', source=source, color=color, legend_label = ticker_name)
    renderers.append(l)

    select.line('Date_', 'RelativeChange_', source=source, color=color)

graph.legend.location = 'top_left'

hover = HoverTool(tooltips=[("Crypto name: ", "@Legend_label_"),("Date: ", "@Date_{%F}"), ("Price change: ", "@RelativeChange_{0.000}")],
                   renderers=renderers, formatters={'@Date_': 'datetime'})
graph.add_tools(hover)
select.ygrid.grid_line_color=None
select.add_tools(range_tool)

show(column(graph,select))

/tmp/ipykernel_25476/4188830622.py:13: UserWarning: no explicit representation of timezones available for np.datetime64
  dates = np.array(tickers['BTC-EUR']['Date'], dtype=np.datetime64)


Some sandboxing with how the RangeTool works. Will probably omit from the actual visualization

In [ ]:
# https://www.geeksforgeeks.org/adding-tooltips-to-a-timeseries-chart-hover-tool-in-python-bokeh/
import pandas as pd
import numpy as np
from bokeh.models import RangeTool, ColumnDataSource
from bokeh.plotting import figure, show, column
data = yf.download('SPY')
data.reset_index(inplace=True)

dates = np.array(data['Date'], dtype=np.datetime64)

data['RelativeChange'] = data.Close / data.Close.iloc[0]
source = ColumnDataSource(data)

p = figure(height=300, width=1000,  x_axis_type = "datetime", title = "Relative price changes", tools='xpan',
           toolbar_location = None, x_axis_location = "above", x_range =(dates[1500], dates[2500]))

p.line('Date_', 'RelativeChange_', source=source)
p.yaxis.axis_label="Relative Price"

select = figure(title="Drag the middle and edges of the selection box to change the range",
                height=150, width=1000, y_range=p.y_range, x_axis_type="datetime",
                tools="", toolbar_location=None)

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('Date_', 'RelativeChange_', source=source)

data2 = yf.download('NVDA')
source2 = ColumnDataSource(data2)
p.line('Date', 'Adj Close_NVDA', source=source2) 
select.line('Date', 'Adj Close_NVDA', source=source2)



select.ygrid.grid_line_color=None
select.add_tools(range_tool)
#select.toolbar.active_multi = range_tool

show(column(p,select))

[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_25476/311101863.py:9: UserWarning: no explicit representation of timezones available for np.datetime64
  dates = np.array(data['Date'], dtype=np.datetime64)
[*********************100%***********************]  1 of 1 completed
